# Tutorial to use construct-tracker for creating lexicons with Generative AI


- Author: Daniel M. Low
- License: Apache 2.0
- Date: 29/07/2024

In [1]:
# development mode:
# !pip uninstall -y construct-tracker
# !pip install git+https://github.com/danielmlow/construct-tracker.git@daniels_branch#egg=construct_tracker&subdirectory=dist

# production mode:
# pip install contruct-tracker
import pandas as pd
import sys
import os
sys.path.append('./../src/')
# sys.path.append('./../src/construct_tracker/')

from construct_tracker import lexicon

In [2]:
# # TODO remove
# from importlib import reload
# reload(lexicon)

In [3]:
documents = ['He is too competitive',
 'Every time I speak with my cousin Bob, I have great moments of insight, clarity, and wisdom',
 "He meditates a lot, but he's not super smart"]

# Create lexicon with generative AI

#

# Add API keys

Each provider has one API key associated to your account. Choose provider and a specific provider's model: https://docs.litellm.ai/docs/providers

**Free trial key** 
- Some providers offer a key for a few submissions per minute for free. 
- Obtain Cohere key: 
	1. Sign in to https://dashboard.cohere.com/api-keys
	2. copy the key under "Trial keys FREE"


**Paid key**
- Obtain OpenAI key: 
	1. Select "Log in" (upper right corner): https://platform.openai.com/
	2. Select "Dashboard" (upper right corner)
	3. Select "API keys" (left panel)
	4. Select "Create new secret key"




# A. Quick start

### 1. Build lexicon with Generative AI and then count in documents

In [6]:
# # TODO: remove commented code

my_lexicon = lexicon.Lexicon()         # Initialize lexicon



manually_added_insight = ['acuity', 'acumen', 'awareness', 'clarity', 'comprehension', 'depth', 'discernment', 'enlightenment', 'epiphany', 'foresight', 'grasp', 'illumination', 'insightfulness', 'intuition', 'knowledge', 'perception', 'perceptiveness', 'perspicacity', 'profoundness', 'realization', 'recognition', 'revelation', 'shrewdness', 'understanding', 'vision', 'wisdom']
my_lexicon.add('Insight', section = 'tokens', value = manually_added_insight)
# my_lexicon.add('Insight', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# 	  examples = ['insight', 'realized'])

manually_added_mindfulness = ['meditate','Buddhism', 'acceptance', 'attentiveness', 'awareness', 'balance', 'being', 'body-scan', 'breath', 'calm', 'centeredness', 'clarity', 'compassion', 'concentration', 'connection', 'contemplation', 'dharma', 'equanimity', 'flow', 'focus', 'gentleness', 'gratitude', 'grounding', 'harmony', 'here and now', 'inner-peace', 'insight', 'intention', 'kindness', 'letting-go', 'mantra', 'meditation', 'mindful', 'mindfulness', 'mindfulness-based', 'non-attachment', 'non-judgment', 'noticing', 'observation', 'openness', 'patience', 'peace', 'presence', 'presence of m', 'quiet', 'reflection', 'relaxation', 'resilience', 'self-awareness', 'self-care', 'serenity', 'silence', 'simplicity', 'stillness', 'surrender', 'tranquility', 'understanding', 'vipassana', 'well-being', 'zen']
my_lexicon.add('Mindfulness', section = 'tokens', value = manually_added_mindfulness)
# my_lexicon.add('Mindfulness', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# 	examples = ['mindfulness', 'meditate'])


manually_added_compassion = ['compassion', 'love', 'kind', 'help others']
my_lexicon.add('Compassion', section = 'tokens', value = manually_added_compassion)
# my_lexicon.add('Compassion', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# 	examples = ['compassion', 'love', 'kind', 'help others'])


Adding 'Insight' to the lexicon (it was not previously there). If you meant to add to an existing construct (and have a typo in the construct name, which is case sensitive), run: del your_lexicon_name.constructs['Insight']
Adding 'Mindfulness' to the lexicon (it was not previously there). If you meant to add to an existing construct (and have a typo in the construct name, which is case sensitive), run: del your_lexicon_name.constructs['Mindfulness']
Adding 'Compassion' to the lexicon (it was not previously there). If you meant to add to an existing construct (and have a typo in the construct name, which is case sensitive), run: del your_lexicon_name.constructs['Compassion']


In [7]:
construct = 'insight'
print(f"Adding '{construct}' to the lexicon (it was not previously there). If you meant to add to an existing construct"
					f" (and have a typo in the construct name, which is case sensitive), run: \n del your_lexicon_name.constructs[{construct}]")

Adding 'insight' to the lexicon (it was not previously there). If you meant to add to an existing construct (and have a typo in the construct name, which is case sensitive), run: 
 del your_lexicon_name.constructs[insight]


In [8]:
print(my_lexicon.constructs['Insight']['tokens'])

print(my_lexicon.constructs['Mindfulness']['tokens'])


['acuity', 'acumen', 'awareness', 'clarity', 'comprehension', 'depth', 'discernment', 'enlightenment', 'epiphany', 'foresight', 'grasp', 'illumination', 'insightfulness', 'intuition', 'knowledge', 'perception', 'perceptiveness', 'perspicacity', 'profoundness', 'realization', 'recognition', 'revelation', 'shrewdness', 'understanding', 'vision', 'wisdom']
['Buddhism', 'acceptance', 'attentiveness', 'awareness', 'balance', 'being', 'body-scan', 'breath', 'calm', 'centeredness', 'clarity', 'compassion', 'concentration', 'connection', 'contemplation', 'dharma', 'equanimity', 'flow', 'focus', 'gentleness', 'gratitude', 'grounding', 'harmony', 'here and now', 'inner-peace', 'insight', 'intention', 'kindness', 'letting-go', 'mantra', 'meditate', 'meditation', 'mindful', 'mindfulness', 'mindfulness-based', 'non-attachment', 'non-judgment', 'noticing', 'observation', 'openness', 'patience', 'peace', 'presence', 'presence of m', 'quiet', 'reflection', 'relaxation', 'resilience', 'self-awareness',

In [9]:
# Now count whether tokens appear in document:

counts, matches_by_construct, matches_doc2construct, matches_construct2doc  = my_lexicon.extract(documents,
                                                                                      normalize = False,
                                                                                      )
display(counts)

extracting... 


100%|██████████| 3/3 [00:02<00:00,  1.29it/s]


,document_id,document,Insight,Mindfulness,Compassion,word_count
0,0,He is too competitive,0,0,0,4
1,1,"Every time I speak with my cousin Bob, I have ...",2,2,0,17
2,2,"He meditates a lot, but he's not super smart",0,1,0,8


In [10]:
# Interpret counts: visualize matches in context  
show_n_sentences = 2
construct = 'Mindfulness'
print(f'Matches for {construct}:')
lexicon.highlight_matches(documents, construct,show_n_sentences, matches_construct2doc)
print()


construct = 'Insight'
print(f'Matches for {construct}:')
lexicon.highlight_matches(documents, construct,show_n_sentences, matches_construct2doc)
print()

Matches for Mindfulness:



Matches for Insight:


### 2. Construct-Text Similarity (CTS)


In [12]:
from construct_tracker import cts

from importlib import reload
reload(cts)

<module 'construct_tracker.cts' from '/Users/danielmlow/Dropbox (MIT)/datum/construct-tracker/tutorials/./../src/construct_tracker/cts.py'>

In [17]:
from construct_tracker import cts
lexicon_dict = my_lexicon.to_dict()
features, documents_tokenized, lexicon_dict_final_order, cosine_similarities = cts.measure(
    lexicon_dict,
    documents,
	count_if_exact_match = False, # False: for now, do not add exact matches to similarities nor replace similarities for exact matches when found
	verbose = True # True: Log information for each step 
    )

display(features.round(2))


Default input sequence length for all-MiniLM-L6-v2: 256
Encoding 180 new construct tokens...


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Finished encoding construct tokens.
Tokenizing documents...
Finished tokenization.
Encoding all document tokens...
computing similarity between 3 constructs and 3 documents...
3it [00:00, 415.99it/s]
Finished.


,document_id,documents,documents_tokenized,Insight_max,Mindfulness_max,Compassion_max
0,0,He is too competitive,[He is too competitive],0.00,0.00,0.0
1,1,"Every time I speak with my cousin Bob, I have ...","[Every time I speak with my cousin Bob, I have...",0.45,0.38,0.0
2,2,"He meditates a lot, but he's not super smart","[He meditates a lot, he's not super smart]",0.31,0.72,0.0


In [15]:
# Interpret scores
doc_id = 2
construct = 'Mindfulness'
most_similar_lexicon_token, most_similar_document_token, highest_similarity = cts.get_highest_similarity_phrase(doc_id, construct, documents, documents_tokenized, cosine_similarities, lexicon_dict_final_order)

The construct 'Mindfulness' through its token 'meditate' had the highest cosine similarity (0.72) with the following document token:
'He meditates a lot'


# B. Use all special features

## 1. Build lexicon with Generative AI 


Specify all the constructs you'd like to measure below. We'll show an example for Insight, Mindfulness, and Compassion

In [6]:
google_drive = False # Load files from Google Drive

if google_drive:
	# Sign into drive to gain access to your documents in a dataframe and/or api_keys.py
	from google.colab import drive
	drive.mount('/content/drive')
	sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [8]:
# See explanation and instructions in Quick Start above. 
import api_keys # local api_keys.py file (where there are variables open_ai='MY-API-KEY') or add string here.

os.environ["OPENAI_API_KEY"]  = api_keys.open_ai  # private API key
gpt4o = "gpt-4o-2024-05-13" # model name
gpt4o_mini = "gpt-4o-mini-2024-07-18"

# Select a model you'll use for now:
model = gpt4o_mini

In [9]:
my_lexicon = lexicon.Lexicon()			# Initialize lexicon
my_lexicon.name = 'Insight'		# Set lexicon name
my_lexicon.description = 'Insight lexicon with constructs inspired by items of the Emotional Insight Scale'
my_lexicon.creator = 'DML' 				# your name or initials for transparency in logging who made changes
my_lexicon.version = '1.0'				# Set version. Over time, others may modify your lexicon, so good to keep track. MAJOR.MINOR. (e.g., MAJOR: new constructs or big changes to a construct, Minor: small changes to a construct)

In [10]:
# Fill out this information
domain = 'psychology' # Optional: bias definitions to a certain domain. this can be "mental health", "economics" or set to None (no quotation marks)
models_for_lexicon = [gpt4o] # Here you could add more: [gpt4o,command_nightly]
model_for_definition = gpt4o # You can ask model to create definition for each construct in the lexicon. Or set to None (no quotation marks)
temperatures = [0,0.5,1] # Temperature is another important parameter that defines how creative a model should be (0: a model outputs the most probable tokens given the prompt and training data; 1=more creative and less predictable responses, with a maximum of 2, which is not often used).

### B.2. Add examples and definitions manually or using a modemy_lexicon (OPTIONAL) 

**Examples**

Add a few prototypical tokens you don't want to miss. With these examples, help guide the model as to whether you want adjectives, nouns, phrases or all of the above. 


**Definitions**

We recommend this for expert validation (step after creating the lexicon, so raters can decide whether to include a token as a function of a specific definition). However, you can decide whether the Generative AI model sees the definition or not so as to guide or not guide the lexicon creation.

We'll show an example where we manually provided a definition (insight), where we'll have our model provide it, and where we will not include it. 

In [31]:
# Fill in

construct_information = {
	'Insight': {
		'examples': ['clarity', 'enlightenment', 'wise'],
		'definition': "the clarity of understanding of one's thoughts, feelings and behavior",
		'reference': "Grant, A. M., Franklin, J., & Langford, P. (2002). The self-reflection and insight scale: A new measure of private self-consciousness. Social Behavior and Personality: an international journal, 30(8), 821-835."
			 },
	'Mindfulness': {
		'examples': ['mindful','meditation', 'awareness', 'nonjudgemental', 'present-focused'],
		'definition': model_for_definition, # it will later check if definition == model_for_definition, if it does, it will generate using that model
		'reference':model_for_definition
			 },	
	'Compassion': {
		'examples':['compassion', 'love', 'kind', 'help others'],
		'definition': None,
		'reference':None
			 },
}

## Definition will be added if model_for_definition is not None using our own prompt. Or you can can directly prompt APIs here with your own prompt. This is our default one:
# prompt = f"Provide a brief definition of {construct} (in the {domain} domain). Provide reference in APA format where you got it from. Return result in the following format: {'{construct}': 'the_definition', 'reference':'the_reference'}"
# definition = api_request(prompt, model = model)

In [32]:
# # # TODO: remove commented code

# my_lexicon = lexicon.Lexicon()         # Initialize lexicon



# manually_added_insight = ['acuity', 'acumen', 'awareness', 'clarity', 'comprehension', 'depth', 'discernment', 'enlightenment', 'epiphany', 'foresight', 'grasp', 'illumination', 'insightfulness', 'intuition', 'knowledge', 'perception', 'perceptiveness', 'perspicacity', 'profoundness', 'realization', 'recognition', 'revelation', 'shrewdness', 'understanding', 'vision', 'wisdom']
# my_lexicon.add('Insight', section = 'tokens', value = manually_added_insight)
# # my_lexicon.add('Insight', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# # 	  examples = ['insight', 'realized'])

# manually_added_mindfulness = ['meditate','Buddhism', 'acceptance', 'attentiveness', 'awareness', 'balance', 'being', 'body-scan', 'breath', 'calm', 'centeredness', 'clarity', 'compassion', 'concentration', 'connection', 'contemplation', 'dharma', 'equanimity', 'flow', 'focus', 'gentleness', 'gratitude', 'grounding', 'harmony', 'here and now', 'inner-peace', 'insight', 'intention', 'kindness', 'letting-go', 'mantra', 'meditation', 'mindful', 'mindfulness', 'mindfulness-based', 'non-attachment', 'non-judgment', 'noticing', 'observation', 'openness', 'patience', 'peace', 'presence', 'presence of m', 'quiet', 'reflection', 'relaxation', 'resilience', 'self-awareness', 'self-care', 'serenity', 'silence', 'simplicity', 'stillness', 'surrender', 'tranquility', 'understanding', 'vipassana', 'well-being', 'zen']
# my_lexicon.add('Mindfulness', section = 'tokens', value = manually_added_mindfulness)
# # my_lexicon.add('Mindfulness', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# # 	examples = ['mindfulness', 'meditate'])


# manually_added_compassion = ['compassion', 'love', 'kind', 'help others']
# my_lexicon.add('Compassion', section = 'tokens', value = manually_added_compassion)
# # my_lexicon.add('Compassion', section = 'tokens', value = 'create', source = model, max_tokens = 150,
# # 	examples = ['compassion', 'love', 'kind', 'help others'])


In [33]:
# # prompt = f"""Please provide a brief definition of {construct} (in the {domain} domain). Provide reference in APA format where you got it from. Return result in the following string format:\n{construct}: the_definition\nreference: the_reference"""
# prompt = f"""Please provide a brief definition of {construct} (in the {domain} domain). Provide reference in APA format where you got it from. Return result in the following string format:
# 			{construct}: the_definition
# 			reference: the_reference"""

# definition = api_request(prompt, model = model_for_definition)
# definition

In [34]:
construct_dict['definition']

In [35]:
construct_dict['definition'] ==  None

True

In [36]:
for construct, information in construct_information.items():
	print('\n',construct, ' ------------------------')
	examples, definition, reference = information['examples'], information['definition'], information['reference']
	
	# DEFINITION: add definition or use model to get one
	# ===================================================
	construct_dict = construct_information.get(construct)
	# if definition exists (is str), use it.
	if construct_dict['definition'] == model_for_definition:
		# Use model to create a definition
		definition, reference = my_lexicon.generate_definition(construct, model = model_for_definition, domain=domain, timeout=45, num_retries=2)
		print(f'Generated definition: {definition}\n')
		print(f'Reference: {reference}\n')
		# Update with definition
		construct_information[construct] = {'definition': definition, 'reference': reference}
	elif isinstance(construct_dict['definition'], str):
		definition = construct_dict['definition']
		reference = construct_dict['reference']
	elif construct_dict['definition'] == None:
		definition = None
		reference = None

	# PROMPT: add definition and examples to prompt
	# ===================================================
	prompt = lexicon.generate_prompt(construct,
                         prompt_name=construct,
                         domain = domain,
						 definition = definition,
						 examples = examples)
	
	print('Prompt used:')
	print(prompt)
	print()

	## Or create your own prompt here:
	# prompt = """
	# Create a list of words and phrases related to {construct} in the {domain} domain. Here are some examples: {examples}
	# """
	# prompt.format(construct = construct, domain = domain, example = examples)
	# print(prompt)
	
	# Generate lexicon
	# ===================================================
	# Each model and temperature will create redudant and new tokens. They will merge with other tokens already generated. 
	for model in models_for_lexicon:
		for temperature in temperatures:
			my_lexicon.add(construct, 
				section = 'tokens', 
				value = 'create', 
				prompt = prompt, 
				source = model, 
				temperature = temperature, 
				max_tokens = 150,
				domain = domain, # so this is saved to metadata
				examples = examples, # so this is saved to metadata
				definition = definition, # so this is saved to metadata
				definition_references = reference, # so this is saved to metadata
				)
	print('Models used to generate tokens:')
	[print(n) for n in list(my_lexicon.constructs[construct]['tokens_metadata'].keys())]
	print()
	print('Generated lexicon:', my_lexicon.constructs[construct]['tokens']) 
	print()





 Insight  ------------------------
Prompt used:
Provide many single words and some short phrases (all in lower case unless the word is generally in upper case) related to Insight (in the psychology domain). Each token should be separated by a semicolon. Do not return duplicate tokens. Do not provide any explanation or additional text beyond the tokens.
Here is a definition of Insight: the clarity of understanding of one's thoughts, feelings and behavior
Here are some examples (include these in the list): clarity; enlightenment; wise

Models used to generate tokens:
gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-45-09.386548
gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_tokens-150, seed-42, 24-08-08T20-45-11.914835
gpt-4o-2024-05-13, temperature-1, top_p-1, max_tokens-150, seed-42, 24-08-08T20-45-14.147452
gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-50-11.974527
gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_toke

Adding 'Compassion' to the lexicon (it was not previously there). If you meant to add to an existing construct (and have a typo in the construct name, which is case sensitive), run: del your_lexicon_name.constructs['Compassion']


Models used to generate tokens:
gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-08.444721
gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-11.147715
gpt-4o-2024-05-13, temperature-1, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-13.810209
gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-09.026274
gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-12.323891
gpt-4o-2024-05-13, temperature-1, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-15.051547

Generated lexicon: ['acceptance', 'attention', 'attentiveness', 'attunement', 'awaken', 'awareness', 'awareness training', 'balance', 'being', 'body scan', 'body-scan', 'breath', 'breathing', 'calm', 'centeredness', 'centering', 'clarity', 'cognitive defusion', 'cognitive-flexibility', 'compassion', 'concentration', 'consciousness', 'contemplation', 'curiosity', 'deliberate', 'detachment', 'dharma', 'e


### 1.2. Important: review resulting lexicon. Add or remove tokens manually.

##### Option A: using python

In [37]:
for construct in construct_information.keys():
	print(f"{construct}:", my_lexicon.constructs[construct]['tokens'])




Insight: ['acuity', 'acumen', 'analysis', 'analyzing', 'apprehension', 'astuteness', 'awaken', 'awareness', 'behavioral insight', 'brilliance', 'clarity', 'clarity of mind', 'cognition', 'cognitive', 'cognitive clarity', 'comprehension', 'consciousness', 'contemplation', 'deep understanding', 'depth', 'detection', 'diagnostic', 'discerned', 'discernment', 'discovery', 'elucidation', 'emotional insight', 'enlightenment', 'epiphany', 'fathoming', 'foresight', 'grasp', 'grasping', 'illumination', 'inner awareness', 'inner clarity', 'inner perception', 'inner realization', 'inner vision', 'inner wisdom', 'insightfulness', 'intelligence', 'interpretation', 'introspection', 'intuition', 'intuitive grasp', 'intuitive understanding', 'intuitiveness', 'judgment', 'keenness', 'knowing', 'knowledge', 'learning', 'lucidity', 'mental clarity', 'mindfulness', 'observance', 'observancy', 'observation', 'penetration', 'percept', 'perception', 'perceptive', 'perceptiveness', 'perceptivity', 'perspectiv

Add or remove tokens in a list that definitely should /shouldn't be there. BE CAREFUL WITH TYPOS OR MISPELLINGS. 

In source you can add any additional description. Recommend putting your initials as well available in `my_lexicon.creator`. 

In [38]:
my_lexicon.add('Mindfulness', section ='tokens',value = ['meditate', 'pay attention'], source=my_lexicon.creator + ": added a few verb forms") # Up to you as an expert
my_lexicon.remove('Mindfulness', remove_tokens = ['being', 'flow'], source =my_lexicon.creator + ": might capture non-compassion realted context too often") # Up to you as an expert



Now those appear as entries in the metadata. All additions are merged and all removed are removed

In [48]:
for n in my_lexicon.constructs['Mindfulness']['tokens_metadata'].keys():
	print('action: ', 
	my_lexicon.constructs['Mindfulness']['tokens_metadata'][n]['action'],
	'from source: ',
	n)
	

action:  create from source:  gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-08.444721
action:  create from source:  gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-11.147715
action:  create from source:  gpt-4o-2024-05-13, temperature-1, top_p-1, max_tokens-150, seed-42, 24-08-08T20-52-13.810209
action:  create from source:  gpt-4o-2024-05-13, temperature-0, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-09.026274
action:  create from source:  gpt-4o-2024-05-13, temperature-0.5, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-12.323891
action:  create from source:  gpt-4o-2024-05-13, temperature-1, top_p-1, max_tokens-150, seed-42, 24-08-08T20-54-15.051547
action:  manually added from source:  DML: added a few verb forms 24-08-08T21-13-27.548503
action:  remove from source:  DML: might capture non-compassion realted context too often 24-08-08T21-13-27.571874


##### Option B: export to csv, modify spreadsheat and import back into python

```python
output_dir = f'./data/lexicons/{my_lexicon.name.replace(" ", "-")}/preprocessing/'
os.makedirs(output_dir, exist_ok=True)
my_lexicon.save(output_dir)
```

In [36]:
# TODO: test


This outputs, several files including `Insight_24-08-01T21-07-37.681390_annotation.csv`. Where you can add tokens to the lexicon in the `<construct_name>_add` column and remove by putting a `0` in the `<construct_name>_include` column. 

```python
from lexicon import add_remove_from_annotation_file

output_dir = ''
filename = "Insight_24-08-01T21-07-37.681390_annotation.csv"

lexicon_annotation_df = pd.read_csv(output_dir+filename, index_col=0)
lexicon_annotation_df = lexicon_annotation_df.reset_index(drop=True)
# Sometimes encoding might be off. Try encoding="latin1" or pd.read_excel(path, encoding="latin1",sheet_name="Sheet1")

my_lexicon = add_remove_from_annotation_file(lexicon_annotation_df, my_lexicon)
```

### 1.3. Confirm additions and removals were successful. 

`['tokens']` contains the final tokens. 



In [49]:
for construct in construct_information.keys():
	print(f"{construct}:",my_lexicon.constructs[construct]['tokens'])



Insight: ['acuity', 'acumen', 'analysis', 'analyzing', 'apprehension', 'astuteness', 'awaken', 'awareness', 'behavioral insight', 'brilliance', 'clarity', 'clarity of mind', 'cognition', 'cognitive', 'cognitive clarity', 'comprehension', 'consciousness', 'contemplation', 'deep understanding', 'depth', 'detection', 'diagnostic', 'discerned', 'discernment', 'discovery', 'elucidation', 'emotional insight', 'enlightenment', 'epiphany', 'fathoming', 'foresight', 'grasp', 'grasping', 'illumination', 'inner awareness', 'inner clarity', 'inner perception', 'inner realization', 'inner vision', 'inner wisdom', 'insightfulness', 'intelligence', 'interpretation', 'introspection', 'intuition', 'intuitive grasp', 'intuitive understanding', 'intuitiveness', 'judgment', 'keenness', 'knowing', 'knowledge', 'learning', 'lucidity', 'mental clarity', 'mindfulness', 'observance', 'observancy', 'observation', 'penetration', 'percept', 'perception', 'perceptive', 'perceptiveness', 'perceptivity', 'perspectiv

### 1.4. Save preliminary lexicon

In [ ]:
clean_lexicon_name = my_lexicon.name.replace(" ", "-")+'_v'+my_lexicon.version.replace('.', '-')
output_dir = f'./my_lexicons/{clean_lexicon_name}/preprocessing'
os.makedirs(output_dir, exist_ok=True) # TODO: remove, will be in save()
my_lexicon.save(output_dir)

For a given construct, you can review metadata, which contains the history of different actions (creating using Gen AI, adding, removing), which comes with a timestamp, the specific prompt used (including examples, definition, if any). All of these entries 

In [38]:

import json
for construct in construct_information.keys():
	print("==="*30)
	print(construct)
	print()
	print(json.dumps(my_lexicon.constructs[construct]['tokens_metadata'], indent=4))



Insight

{
    "None 24-08-07T21-02-54.555244": {
        "action": "add",
        "tokens": [
            "acuity",
            "acumen",
            "awareness",
            "clarity",
            "comprehension",
            "depth",
            "discernment",
            "enlightenment",
            "epiphany",
            "foresight",
            "grasp",
            "illumination",
            "insightfulness",
            "intuition",
            "knowledge",
            "perception",
            "perceptiveness",
            "perspicacity",
            "profoundness",
            "realization",
            "recognition",
            "revelation",
            "shrewdness",
            "understanding",
            "vision",
            "wisdom"
        ]
    }
}
Mindfulness

{
    "None 24-08-07T21-02-54.576558": {
        "action": "add",
        "tokens": [
            "Buddhism",
            "acceptance",
            "attentiveness",
            "awareness",
            "balan

Also available in _metadata.json file when saving the lexicon



### 1.5. Extract counts

Calibration step: this can be done on a examples you come up with or on training set to make sure it's doing a good job

In [55]:
# Now count whether tokens appear in document:

counts, matches_by_construct, matches_doc2construct, matches_construct2doc  = my_lexicon.extract(documents,
                                                                                      normalize = False,
                                                                                      )
display(counts)

extracting... 


100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


,document_id,document,Insight,Mindfulness,Compassion,word_count
0,0,He is too competitive,0,0,0,4
1,1,"Every time I speak with my cousin Bob, I have ...",2,2,0,17
2,2,"He meditates a lot, but he's not super smart",0,1,0,8


In [60]:
# Interpret counts: visualize matches in context  

show_n_sentences = 2
for construct in construct_information.keys():
	print(f'------- Matches for {construct}:')
	lexicon.highlight_matches(documents, construct,show_n_sentences, matches_construct2doc)
	print()


------- Matches for Insight:------- 



------- Matches for Mindfulness:------- 



------- Matches for Compassion:------- 



In [ ]:
# TODO: other code I used to look at context


### 1.6. Validate with experts or just crowdsourcing (OPTIONAL)

1. Create `human_ratings/` folder

In [ ]:
output_dir = f'./my_lexicons/{clean_lexicon_name}/human_ratings/' 
os.makedirs(output_dir, exist_ok=True)

2. Take the `Insight_24-08-08T21-29-52_annotation.csv` in the `preprocessing/` directory and send a copy to each human rater (aka coder, annotator) with their ID appended to the filename.

In [72]:
# I'll create three duplicates of the annotations file identifying each file with raters initiails
import shutil # TODO move up
rater_ids = ['MKH', 'KHB', 'DLC'] 

# # or use IDs:
# n_raters = 3
# rater_ids = [str(i).zfill(3) for i in range(n_raters)] # ['001', '002', '003']

path_to_unvalidated_lexicon = f'./my_lexicons/{clean_lexicon_name}/preprocessing/'+'Insight_24-08-08T21-29-52_ratings.csv'

for rater_id in rater_ids:
	new_path = path_to_unvalidated_lexicon.replace('preprocessing', 'human_ratings').replace('.csv', f'_{rater_id}.csv')
	# copy from preprocessing to human_raters
	shutil.copy(path_to_unvalidated_lexicon, new_path)

# Now you have this file in human_ratings Insight_24-08-08T21-29-52_ratings_DLC.csv

3. Have them rate following these or similar instructions. Save files as csv instead of excel to use the code below

In [ ]:

# TODO

4. Take average ratings, discard tokens below a certain average

In [ ]:
from construct_tracker.lexicon import avg_above_thresh

# TODO: take 

In [ ]:
# TODO
filename = "Insight_24-08-01T21-07-37.681390_annotation_AVG_ANNOTATIONS.csv"

from lexicon import add_remove_from_annotation_file

# TODO: function to average annotations. Have argument option not to add their suggestions. 


lexicon_annotation_df = pd.read_csv(output_dir+filename, index_col=0)
lexicon_annotation_df = lexicon_annotation_df.reset_index(drop=True)
# Sometimes encoding might be off. Try encoding="latin1" or pd.read_excel(path, encoding="latin1",sheet_name="Sheet1")

my_lexicon = add_remove_from_annotation_file(lexicon_annotation_df, my_lexicon)


4. Inter-rater reliability (OPTIONAL)



In [74]:
from construct_tracker.utils import irr

# TODO: See inter_rater_reliability.ipynb 


### 1.7. Save final version




In [45]:
output_dir = f'./my_lexicons/{clean_lexicon_name}/'
os.makedirs(output_dir, exist_ok=True) # TODO: remove, will be in save()
my_lexicon.save(output_dir) # save lexicon


### 1.7. final extraction

If you want to load lexicon in a different script in the future:
```python
path = f'./my_lexicons/{clean_lexicon_name}/'
my_lexicon = load_lexicon(path)
```

In [ ]:
# Now count whether tokens appear in document:

# We'll set normalize to True. Whether to normalize the extracted features by word count. 3 matches in a short document would be weighed higher than in a long document.
counts, matches_by_construct, matches_doc2construct, matches_construct2doc  = my_lexicon.extract(documents,
                                                                                      normalize = True,
                                                                                      )
display(counts)

## 2. Construct Text Similarity

### 2.1. Load validated lexicon or create your a quick one with some keywords

In [41]:
# production mode:
# pip install contruct-tracker

# development mode:
# !pip uninstall -y construct-tracker
# !pip install git+https://github.com/danielmlow/construct-tracker.git@daniels_branch#egg=construct_tracker&subdirectory=dist

# Local mode
import sys
sys.path.append('./../src/')
sys.path.append('./../src/construct_tracker/')
from construct_tracker import lexicon

# Mount drive
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# sys.path.append('/content/drive/My Drive/Colab Notebooks')


In [46]:
lexicon_dict = my_lexicon.to_dict() 



"\n# Or just create your own quickly with some keywords\nlexicon_dict = {\n\t'Insight': ['clarity', 'enlightenment', 'wise'],\n\t'Mindfulness': ['mindful','meditation', 'awareness', 'nonjudgemental', 'present-focused'],\n\t'Compassion': ['compassion', 'love', 'kind', 'help others'],\n\n}\n"


Or Load lexicon
```python
path = './my_lexicons/Insight/Insight_24-08-07T21-11-46.539980.pickle'
my_lexicon = lexicon.load_lexicon(path)
```

Or just create your own quickly with some keywords
```python
lexicon_dict = {
 	'Insight': 	['clarity', 'enlightenment', 'wise'],
 	'Mindfulness': 	['mindful','meditation', 'awareness', 'nonjudgemental',
			'present-focused'],
 	'Compassion': 	['compassion', 'love', 'kind', 'help others'],

 }
 ```

### 2.2. Measure construct-text similarity

In [97]:
# TODO: remove
# from construct_tracker import cts
# from importlib import reload
# reload(cts)


<module 'construct_tracker.cts' from '/Users/danielmlow/Dropbox (MIT)/datum/construct-tracker/tutorials/./../src/construct_tracker/cts.py'>

In [101]:
from construct_tracker import cts

features, documents_tokenized, lexicon_dict_final_order, cosine_similarities = cts.measure(
    lexicon_dict,
    documents,
	count_if_exact_match = 'sum',
	similarity_threshold = 0.3,
    )

display(features)


2024-08-07 18:12:37,466 - construct_tracker.cts - WARNING - Extracting all lexicon token embeddings for scratch...
2024-08-07 18:12:38,172 - construct_tracker.cts - INFO - Default input sequence length for all-MiniLM-L6-v2: 256
INFO:construct_tracker.cts:Default input sequence length for all-MiniLM-L6-v2: 256
2024-08-07 18:12:38,185 - construct_tracker.cts - INFO - Encoding 176 new tokens
INFO:construct_tracker.cts:Encoding 176 new tokens
Batches: 100%|██████████| 6/6 [00:00<00:00,  6.03it/s]
2024-08-07 18:12:39,256 - construct_tracker.cts - INFO - Finished.
INFO:construct_tracker.cts:Finished.
2024-08-07 18:12:39,258 - construct_tracker.cts - INFO - Tokenizing documents...
INFO:construct_tracker.cts:Tokenizing documents...
2024-08-07 18:12:39,936 - construct_tracker.cts - INFO - Finished tokenization.
INFO:construct_tracker.cts:Finished tokenization.
2024-08-07 18:12:39,939 - construct_tracker.cts - INFO - Encoding all document tokens...
INFO:construct_tracker.cts:Encoding all documen

extracting... 


  0%|          | 0/3 [00:00<?, ?it/s]/Users/danielmlow/Dropbox (MIT)/datum/construct-tracker/tutorials/./../src/construct_tracker/lexicon.py:775: UserWarning: Lemmatizing the tokens. We recommend you lemmatize before extracting so you can save time if you want to repeat extraction on different documents.
  warnings.warn(
100%|██████████| 3/3 [00:02<00:00,  1.39it/s]


,document_id,documents,documents_tokenized,Insight_max,Mindfulness_max,Compassion_max
0,0,He is too competitive,[He is too competitive],0.235573,0.219162,0.000000
1,1,"Every time I speak with my cousin Bob, I have ...","[Every time I speak with my cousin Bob, I have...",2.447937,2.382439,0.230919
2,2,"He meditates a lot, but he's not super smart","[He meditates a lot, he's not super smart]",0.312533,1.715475,0.268710


You can also provide a counts DF done in a different way:

```python
counts, matches_by_construct, matches_doc2construct, matches_construct2doc  = my_lexicon.extract(documents,normalize = True,return_zero=['anger'])

from construct_tracker import cts

features, documents_tokenized, lexicon_dict_final_order, cosine_similarities = cts.measure(
    lexicon_dict,
    documents,
	lexicon_counts = counts,
	count_if_exact_match = 'sum',
	similarity_threshold = 0.3,
    )

display(features)
```


### 2.3. Interpret cosine scores for any given document


In [105]:

doc_id = 2
construct = 'Mindfulness'
most_similar_lexicon_token, most_similar_document_token, highest_similarity = cts.get_highest_similarity_phrase(doc_id, construct, documents, documents_tokenized, cosine_similarities, lexicon_dict_final_order)

The construct 'Mindfulness' through its token 'meditate' had the highest cosine similarity (0.72) with the following document token:
'He meditates a lot'


## 3. lbl2vec

Is a similar package, that takes the average of the lexicon and computes the similarity with the entire document (less precise than CTS, but faster) 

https://github.com/sebischair/**Lbl2Vec**


In [ ]:

!pip install lbl2vec

In [ ]:
from lbl2vec import Lbl2TransformerVec
from sentence_transformers import SentenceTransformer

# select sentence-tranformers model
transformer_model = SentenceTransformer('all-MiniLM-L6-v2') #"all-mpnet-base-v2"

# init model
model = Lbl2TransformerVec(transformer_model=transformer_model,
                           keywords_list=list(lexicon_dict.values()), # iterable list of lists with descriptive keywords of type str
                           documents=documents, #iterable list of strings
                           label_names = list(lexicon_dict.keys())
                           )

# train model
model.fit()

similarity_df = model.predict_model_docs() # Predict label similarities for documents used for training
# similarity_df = model.predict_model_docs(doc_idxs = [0,1]) # Predict label similarities for documents used for training

similarity_df


model.save('model_name')
model = Lbl2Vec.load('model_name')





2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-07-29 20:54:03,952 - Lbl2TransformerVec - INFO - Compute keyword embeddings
INFO:Lbl2TransformerVec:Compute keyword embeddings
2024-07-29 20:54:04,137 - Lbl2TransformerVec - INFO - Compute document embeddings
2024-07-29 20:54:04,137 - Lbl2TransformerVec - INFO - Compute document embeddings
2024-07-29 20:54:04,137 - Lbl2TransformerVec - INFO - Compute document embeddings
2024-07-29 20:54:04,137 - Lbl2TransformerVec - INFO - C